In [1]:
!pip install scikit-surprise

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 772.0/772.0 kB 4.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for scikit-surprise: filename=scikit_surprise-1.1.3-cp310-cp310-linux_x86_64.whl size=3162990 sha256=b0b43c0b38edbe88d075eff026a3e08a6143a172ac0d55d8f6a17cfb0b08c513
  Stored in directory: /root/.cache/pip/wheels/a5/ca/a8/4e28def53797fdc4363ca4af740db15a9c2f1595ebc51fb445
Successfully built scikit-surprise


In [2]:
from surprise import SVD, SVDpp, NMF
from surprise import Dataset, accuracy, Reader
from surprise.model_selection import cross_validate, train_test_split, GridSearchCV

In [3]:
# Loading dataset
data = Dataset.load_builtin('ml-100k')

Dataset ml-100k could not be found. Do you want to download it? [Y/n] y
Trying to download dataset from https://files.grouplens.org/datasets/movielens/ml-100k.zip...
Done! Dataset ml-100k has been saved to /root/.surprise_data/ml-100k


In [4]:
# Use the famous SVD algorithm
model = SVD()

# Run 5-fold cross-validation and print results
results_SVD = cross_validate(model, data, measures=['RMSE', 'MAE'], cv=5, verbose=True)

Evaluating RMSE, MAE of algorithm SVD on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    0.9265  0.9376  0.9398  0.9345  0.9358  0.9348  0.0045  
MAE (testset)     0.7315  0.7355  0.7375  0.7389  0.7367  0.7360  0.0025  
Fit time          1.04    1.01    1.61    1.03    1.02    1.14    0.23    
Test time         0.10    0.18    0.16    0.24    0.12    0.16    0.05    


In [5]:
# We divide the dataset into trainset and testset, test set is made of 25% of the ratings.
trainset, testset = train_test_split(data, test_size=0.25)

# We'll use the famous SVD algorithm.
algo = SVD()

# Train the algorithm on the trainset, and predict ratings for the testset
algo.fit(trainset)
predictions = algo.test(testset)

# Then compute RMSE
accuracy.rmse(predictions)

RMSE: 0.9433


0.9433029186333015

In [6]:
# We determine the best parameters n_epochs, lr_all and reg_all using GridSearchCV for the model SVD
params = {'n_epochs': [5, 10, 15], 'lr_all': [0.002, 0.005, 0.01], 'reg_all': [0.02, 0.1, 0.2]}

svd = SVD()

grid_search = GridSearchCV(SVD, params, measures=['rmse', 'mae'], cv=3)

grid_search.fit(data)

print("Best parameters found:", grid_search.best_params)
print("Best RMSE:", grid_search.best_score['rmse'])
print("Best MAE:", grid_search.best_score['mae'])

Best parameters found: {'rmse': {'n_epochs': 15, 'lr_all': 0.01, 'reg_all': 0.1}, 'mae': {'n_epochs': 15, 'lr_all': 0.01, 'reg_all': 0.1}}
Best RMSE: 0.9350993407914135
Best MAE: 0.7409554266764684


In [7]:
# Let's find the same parameters for the algorithm SVD++ (it takes more than 20 minutes)
svdpp = SVDpp()

grid_search_svdpp = GridSearchCV(SVDpp, params, measures=['rmse', 'mae'], cv=3)
grid_search_svdpp.fit(data)

print("Best parameters found for SVD++:", grid_search_svdpp.best_params)
print("Best RMSE for SVD++:", grid_search_svdpp.best_score['rmse'])
print("Best MAE for SVD++:", grid_search_svdpp.best_score['mae'])

Best parameters found for SVD++: {'rmse': {'n_epochs': 10, 'lr_all': 0.01, 'reg_all': 0.02}, 'mae': {'n_epochs': 10, 'lr_all': 0.01, 'reg_all': 0.02}}
Best RMSE for SVD++: 0.9260243440090671
Best MAE for SVD++: 0.7290413963217993


In [8]:
# Let's find the same parameters for the Non-negative Matrix Factorization (NMF) algorithm
nmf = NMF()

param_nmf = {'n_factors': [5, 10, 15], 'n_epochs': [10, 15], 'reg_pu': [0.02, 0.1, 0.2], 'reg_qi': [0.02, 0.1, 0.2]}
grid_search_nmf = GridSearchCV(NMF, param_nmf, measures=['rmse', 'mae'], cv=3)
grid_search_nmf.fit(data)

print("Best parameters found for NMF:", grid_search_nmf.best_params)
print("Best RMSE for NMF:", grid_search_nmf.best_score['rmse'])
print("Best MAE for NMF:", grid_search_nmf.best_score['mae'])

Best parameters found for NMF: {'rmse': {'n_factors': 10, 'n_epochs': 15, 'reg_pu': 0.1, 'reg_qi': 0.2}, 'mae': {'n_factors': 5, 'n_epochs': 15, 'reg_pu': 0.2, 'reg_qi': 0.2}}
Best RMSE for NMF: 0.963036309251342
Best MAE for NMF: 0.7530540231009727


Let's analyze the metrics that we can observe as a result of code execution:

**RMSE Root Mean Squared Error** (testset): Root Mean Squared Error between actual and predicted ratings on the test data set. The smaller the value, the better.

**MAE Mean Absolute Error** (testset): Mean absolute error between actual and predicted ratings on the test data set. The smaller the value, the better.

**Fit time**: The time required to train the model on each section of data. This is usually measured in seconds.

**Test time**: The time required to predict the ratings for the test data set. This is also usually measured in seconds.

---

*Having read the results, we can say that the SVD and SVD++ methods showed better results than the NMF method.
SVD++ showed the best result, but it needs much more time to train the model and get the necessary data.*